<a href="https://colab.research.google.com/github/Arup3201/SmartFlow-Traffic-Detector/blob/main/notebooks/feature_extraction_of_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt